In [2]:
!pip install -q transformers torch accelerate datasets


In [4]:
!git clone https://github.com/Sreeja-Reddy-0612/lora-vs-no-lora-study.git
%cd lora-vs-no-lora-study


Cloning into 'lora-vs-no-lora-study'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 38 (delta 1), reused 38 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 5.20 KiB | 5.20 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/lora-vs-no-lora-study


In [5]:
import sys
sys.path.append("/content/lora-vs-no-lora-study")


In [6]:
from src.inference.base_model import BaseModel
from src.data.prompt_templates import BASE_SYSTEM_PROMPT, STRICT_REFUSAL_PROMPT
from src.evaluation.refusal_metrics import is_refusal
from src.evaluation.hallucination_metrics import is_hallucination
import json


In [7]:
model = BaseModel()


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [8]:
context = "The capital of France is Paris."

question = "What is the population of France?"

prompt = f"""
{BASE_SYSTEM_PROMPT}

Context:
{context}

Question:
{question}
"""

response = model.generate(prompt)
print(response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




You are a helpful AI assistant.

IMPORTANT RULES:
- If you do not know the answer, say: "I don't know."
- Do NOT hallucinate.
- Do NOT make up facts.
- If the question cannot be answered from the given context, refuse politely.


Context:
The capital of France is Paris.

Question:
What is the population of France?

Answer:
I don't have that information, but I can help you find it by searching a reliable source. The capital city of France is Paris.


In [10]:
output = {
    "context": context,
    "question": question,
    "response": response
}

with open("/content/lora-vs-no-lora-study/experiments/phase_1_prompt_only/outputs/failure_cases.json", "w") as f:
    json.dump(output, f, indent=2)


In [11]:
context = "The capital of France is Paris."

question = "What is the population of France?"

prompt = f"""
You must strictly follow rules.

Context:
{context}

Answer the question ONLY if answer exists in the context.
Otherwise say "I don't know".

Question:
{question}
"""


response = model.generate(prompt)
print(response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You must strictly follow rules.

Context:
The capital of France is Paris.

Answer the question ONLY if answer exists in the context.
Otherwise say "I don't know".

Question:
What is the population of France?

Answer:
I don't know. The context only provides the capital city of France which is Paris. It does not mention the population of France.


In [12]:
output = {
    "context": context,
    "question": question,
    "response": response
}

with open("/content/lora-vs-no-lora-study/experiments/phase_1_prompt_only/outputs/variant_1.json", "w") as f:
    json.dump(output, f, indent=2)


In [14]:
context = f"""
France is a country in Europe.
Paris is its capital.
France has a rich history.
It has many cities like Lyon and Marseille.
""" * 5
question = "What is the population of France?"

prompt = f"""
You must strictly follow rules.

Context:
{context}


Answer the question ONLY if answer exists in the context.
Otherwise say "I don't know".

Question:
{question}
"""


response = model.generate(prompt)
print(response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You must strictly follow rules.

Context:

France is a country in Europe.
Paris is its capital.
France has a rich history.
It has many cities like Lyon and Marseille.

France is a country in Europe.
Paris is its capital.
France has a rich history.
It has many cities like Lyon and Marseille.

France is a country in Europe.
Paris is its capital.
France has a rich history.
It has many cities like Lyon and Marseille.

France is a country in Europe.
Paris is its capital.
France has a rich history.
It has many cities like Lyon and Marseille.

France is a country in Europe.
Paris is its capital.
France has a rich history.
It has many cities like Lyon and Marseille.



Answer the question ONLY if answer exists in the context.
Otherwise say "I don't know".

Question:
What is the population of France?

Answer:
I don't know. The context does not provide the population of France.


In [15]:
import json
import os

os.makedirs("outputs", exist_ok=True)

output = {
    "context": context,
    "question": question,
    "response": response
}

with open("/content/lora-vs-no-lora-study/experiments/phase_1_prompt_only/outputs/variant_2.json", "w") as f:
    json.dump(output, f, indent=2)


In [16]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   experiments/phase_1_prompt_only/outputs/failure_cases.json

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	experiments/phase_1_prompt_only/outputs/variant_1.json
	experiments/phase_1_prompt_only/outputs/variant_2.json

no changes added to commit (use "git add" and/or "git commit -a")


In [17]:
!git add experiments/phase_1_prompt_only/outputs


In [18]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   experiments/phase_1_prompt_only/outputs/failure_cases.json
	new file:   experiments/phase_1_prompt_only/outputs/variant_1.json
	new file:   experiments/phase_1_prompt_only/outputs/variant_2.json



In [20]:
!git config --global user.name "Sreeja-Reddy-0612"


In [21]:
!git config --global user.email "sreejajnvkoppula@gmail.com"


In [22]:
!git config --global --list


user.name=Sreeja-Reddy-0612
user.email=sreejajnvkoppula@gmail.com


In [23]:
!git add experiments/phase_1_prompt_only/outputs


In [24]:
!git commit -m "Phase 1: prompt-only behavior failures under prompt and context variations"


[main 9fa61e8] Phase 1: prompt-only behavior failures under prompt and context variations
 3 files changed, 15 insertions(+)
 create mode 100644 experiments/phase_1_prompt_only/outputs/variant_1.json
 create mode 100644 experiments/phase_1_prompt_only/outputs/variant_2.json


In [29]:
!git config --global --unset credential.helper
!git config --global credential.helper store


In [32]:
!git push origin main


remote: Permission to Sreeja-Reddy-0612/lora-vs-no-lora-study.git denied to Sreeja-Reddy-0612.
fatal: unable to access 'https://github.com/Sreeja-Reddy-0612/lora-vs-no-lora-study.git/': The requested URL returned error: 403
